In [1]:
import pandas as pd
import re
import numpy as np
import math
import os

In [13]:
#dbAMP
dbamp_des = pd.read_excel("raw_data/dbAMPv2.0.xls")
dbamp_seq = pd.read_csv('raw_data/dbamp_raw.csv')
dbamp_seq = dbamp_seq.rename(columns={'ID':'dbAMP_ID'})
dbAMP_df = pd.merge(dbamp_seq,dbamp_des,on='dbAMP_ID')
#DBAASP
DBAASP_df = pd.read_csv('raw_data/peptides-complete1220.csv')
#DRAMP
DRAMP_df = pd.read_excel("raw_data/DRAMP.xlsx")

C:\Users\User\AppData\Local\Temp\ipykernel_10368\551976584.py:7: DtypeWarning: Columns (16,17,23,24,31,32,38,53,54,57,63,64,65,66,67,68,71,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  DBAASP_df = pd.read_csv('raw_data/peptides-complete1220.csv')


In [3]:
DBAASP_df = DBAASP_df[DBAASP_df['COMPLEXITY']=='Monomer']

In [4]:
DBAASP_df.loc[DBAASP_df['TARGET ACTIVITY - CONCENTRATION'].isnull(),['TARGET ACTIVITY - CONCENTRATION']] = '10000'
DBAASP_df.head()

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,PDB NAME,...,UNIRPROT - PRO PEPTIDE,UNIRPROT - URL,ARTICLES - JOURNAL,ARTICLES - YEAR,ARTICLES - VOLUME,ARTICLES - PAGES,ARTICLES - TITLE,ARTICLES - ADDITIONAL,ARTICLES - PUBMED,ARTICLES - AUTHORS
98,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,"Gram+,Gram-,Fungus",Lipid Bilayer,NaN,...,NaN,NaN,J Biol Chem,2004.0,279,12277-12285,A new group of antifungal and antibacterial li...,NaN,http://www.ncbi.nlm.nih.gov/pubmed/14709550,"Shai Y,Avrahami D"
99,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,"Gram+,Gram-,Fungus",Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,"Gram+,Gram-,Fungus",Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,"Gram+,Gram-,Fungus",Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,"Gram+,Gram-,Fungus",Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dbAMP_df = dbAMP_df.rename(columns={
    'sequence' : 'SEQUENCE',
})
dbAMP_df = dbAMP_df[dbAMP_df['Targets'].isnull()==False]
dbAMP_df.head()

,dbAMP_ID,SEQUENCE,activities,experimental evidence,Name,Source,Tax,Uniprot,PDB,Targets
4,dbAMP_00005,AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,"jellyfish, Aurelia aurita&&Aurelin (jellyfish,...","jellyfish, Aurelia aurita&&Aurelia aurita (Moo...","Eukaryota,Metazoa,Cnidaria,Scyphozoa,Semaeosto...",Q0MWV8&& P84891&&Q0MWV8,2LG4,Listeria monocytogenes EGD (MIC=22.64µg/ml)&&G...
23,dbAMP_00024,AAFRGCWTKNYSPKPCL,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,"Ranacyclin-B-AL1 (Frogs, amphibians, animals)&...",Amolops loloensis (Ranidae frogs)&&Amolops lol...,"Eukaryota,Metazoa,Chordata,Amphibia,Anura,Rani...",NaN,NaN,Staphylococcus aureus ATCC 25923 (MIC=51.5µM)&...
26,dbAMP_00027,AAGGVKKPKKAAAAKKSPKKPKKPAAA,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,VK13 (histone derived; reptiles; animals)&&VK1...,"blood plasma, Komodo dragon, Varanus komodoensis",NaN,NaN,NaN,Lactobacillus rhamnosus 7469 (MIC=>64µM)&&Lact...
27,dbAMP_00028,AAGMGFFGAR,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,Urechistachykinins II&&Urechistachykinin-2&&Ur...,Urechis unicinctus,"Annelida, Urechis unicinctus &&Animalia, Annel...",P40752,NaN,[Ref.18570895] GrµM-negative bacteria: Escheri...
29,dbAMP_00030,AAGYPFGIKV,"['AntiGram\xa0+', 'AntiGram&nbsp-', 'Antifungal']",NO,Ishikawain-1,NaN,"Odorrana ishikawae&&Eukaryota,Metazoa,Chordata...",NaN,NaN,Staphylococcus aureus ATCC 6538 &&Staphylococc...


In [6]:
DRAMP_df = DRAMP_df.rename(columns={
    'Sequence' : 'SEQUENCE',
})
DRAMP_df.head()

,DRAMP_ID,SEQUENCE,Sequence_Length,Name,Swiss_Prot_Entry,Family,Gene,Source,Activity,Protein_existence,...,N-terminal_Modification,C-terminal_Modification,Other_Modifications,Stereochemistry,Cytotoxicity,Binding_Traget,Pubmed_ID,Reference,Author,Title
0,DRAMP00005,SLGPAIKATRQVCPKATRFVTVSCKKSDCQ,30,Epicidin 280 (Bacteriocin),O54220,Belongs to the lantibiotic family (Class I bac...,eciA,Staphylococcus epidermidis BN 280 (Gram-positi...,"Antimicrobial, Antibacterial, Anti-Gram+",Protein level,...,Oxypropionylation,Not metioned clearly,There are possible lanthionine(Lan)/3-methylla...,L,No cytotoxicity information found,Not found,9726851,Appl Environ Microbiol. 1998 Sep;64(9):3140-3146.,"Heidrich C, Pag U, Josten M, Metzger J, Jack R...","Isolation, characterization, and heterologous ..."
1,DRAMP00017,VTSWSLCTPGCTSPGGGSNCSFCC,24,Microbisporicin A1 (Bacteriocin),No entry found,Belongs to the lantibiotic family (Class I bac...,Not found,Microbispora corallina (Gram-positive bacteria),"Antimicrobial, Antibacterial",Protein level,...,Free,Amidation and Cyclization,①There are five thioether intramolecular bridg...,L,No cytotoxicity information found,Not found,18215770,Chem Biol. 2008 Jan;15(1):22-31.,"Castiglione F, Lazzarini A, Carrano L, Corti E...",Determining the structure and mode of action o...
2,DRAMP00032,GNGVLKTISHECNMNTWQFLFTCC,24,Ruminococcin A (RumA; Bacteriocin),"P83674, P83676, P83677, Q8VLK0, Q9K381, P83675...",Belongs to the type A lantibiotic family (Clas...,rumA1 AND rumA2 AND,Ruminococcus gnavus & Ruminococcus hansenii (G...,"Antimicrobial, Antibacterial, Anti-Gram+",Protein level,...,Free,Cyclization (possibly),① It is highly probable that (i) Ser9 and the ...,L,No cytotoxicity information found,Cell membrane,11526013##11741840##12089024,Appl Environ Microbiol. 2001 Sep;67(9):4111-41...,"Dabard J, Bridonneau C, Phillipe C, Anglade P,...","Ruminococcin A, a new lantibiotic produced by ..."
3,DRAMP00063,SSSGWLCTLTIECGTIICACR,21,Lantibiotic michiganin-A (Bacteriocin),Q09T02,Belongs to the type B lantibiotic family (Clas...,micA,Clavibacter michiganensis subsp. Michiganensis...,"Antimicrobial, Antibacterial",Protein level,...,Free,Free,All of the threonine residues undergo dehydrat...,L,No cytotoxicity information found,Not found,16957199,Appl Environ Microbiol. 2006 Sep;72(9):5814-5821.,"Holtsmark I, Mantzilas D, Eijsink VG, Brurberg...","Purification, characterization, and gene seque..."
4,DRAMP00068,MSWLNFLKYIAKYGKKAVSAAWKYKGKVLEWLNVGPTLEWVWQKLK...,51,Aureocin A53 (Bacteriocin),Q8GPI4,Belongs to the class II bacteriocin,aucA,Staphylococcus aureus A53 (Gram-positive bacte...,"Antimicrobial, Antibacterial, Anti-Gram+",Protein level,...,Formylation,Free,None,L,No cytotoxicity information found,Cell membrane,12054867,J Mol Biol. 2002 Jun 7;319(3):745-756.,"Netz DJ, Pohl R, Beck-Sickinger AG, Selmer T, ...",Biochemical characterisation and genetic analy...


In [7]:
def Nature_AA(new_df):
    # 僅保留天然胺基酸
    nature = list()
    new_df['NATURE'] = False
    output = False
    for seq in new_df['SEQUENCE']: 
        for char in str(seq):
            if (char == 'R' or char == 'H' or char == 'K' or char == 'D' or char == 'E' or char == 'S' or char == 'T' or char == 'N' or char == 'Q' or char == 'C' or char == 'G' or char == 'P' or char == 'A' or char == 'I' or char == 'L' or char == 'M' or char == 'F' or char == 'W' or char == 'Y' or char == 'V'):
                output = True
            else:
                output = False
                break
        nature.append(output)
    new_df['NATURE'] = nature
    new_df = new_df[new_df['NATURE']==True]
    
    return new_df
def Seq_Length(new_df):
    # 計算每個長度
    new_df['SEQ_LEN'] = new_df['SEQUENCE'].apply(len)
    return new_df

def Seq_Length_cut(new_df):
    #介於6~40之間
    new_df = new_df[new_df['SEQ_LEN']>5]
    new_df = new_df[new_df['SEQ_LEN']<41]
    
    return new_df

def Cal_weight(new_df):
    # 計算原子重量
    weights = {'A': 71.08, 'C': 103.15, 'D': 115.09, 'E': 129.12, 'F': 147.18,
           'G': 57.05, 'H': 137.14, 'I': 113.16, 'K': 128.18, 'L': 113.16,
           'M': 131.20, 'N': 114.11, 'P': 97.12, 'Q': 128.13, 'R': 156.19,
           'S': 87.08, 'T': 101.11, 'V': 99.13, 'W': 186.22, 'Y': 163.18 }

    new_df['MW']= new_df['SEQUENCE'].map(lambda seq: sum(weights[p] for p in seq))
    
    return new_df

def Data_process(new_df):
    new_df = Nature_AA(new_df)
    new_df = Seq_Length(new_df)
    new_df = Seq_Length_cut(new_df)
    new_df = Cal_weight(new_df)
    return new_df

In [8]:
DBAASP_df = Data_process(DBAASP_df)
dbAMP_df = Data_process(dbAMP_df)
DRAMP_df = Data_process(DRAMP_df)

dbAMP_df = dbAMP_df.reset_index(drop=True)
DRAMP_df = DRAMP_df.reset_index(drop=True)

# strain = ['Enterococcus faecium', 'Staphylococcus aureus', 'Klebsiella pneumoniae', 'Acinetobacter baumannii',
#           'Pseudomonas aeruginosa', 'Enterobacter spp.', 'Escherichia coli']
strain = ['Staphylococcus aureus','Pseudomonas aeruginosa','Escherichia coli']

C:\Users\User\AppData\Local\Temp\ipykernel_10368\2369554771.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SEQ_LEN'] = new_df['SEQUENCE'].apply(len)
C:\Users\User\AppData\Local\Temp\ipykernel_10368\2369554771.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['SEQ_LEN'] = new_df['SEQUENCE'].apply(len)
C:\Users\User\AppData\Local\Temp\ipykernel_10368\2369554771.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [9]:
DBAASP_df.head()

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,PDB NAME,...,ARTICLES - YEAR,ARTICLES - VOLUME,ARTICLES - PAGES,ARTICLES - TITLE,ARTICLES - ADDITIONAL,ARTICLES - PUBMED,ARTICLES - AUTHORS,NATURE,SEQ_LEN,MW
109,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,1994.0,91,11138-11142,"""Libraries from libraries"": chemical transform...",NaN,http://www.ncbi.nlm.nih.gov/pubmed/7972024,"Ostresh JM,Husar GM,Blondelle SE,Dörner B,Webe...",True,6,815.04
110,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6,815.04
111,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6,815.04
112,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6,815.04
113,11,Monomer,"Cathelicidin-1, CATH-1, Fowlicidin-1",NaN,RVKRVWPLVIRTVIAGYNLYRAIKKK,NaN,Ribosomal,"Gram+,Gram-,Cancer,Mammalian Cell",Lipid Bilayer,2AMN,...,2006.0,273,2581-2593,Structure-activity relationships of fowlicidin...,NaN,http://www.ncbi.nlm.nih.gov/pubmed/16817888,"Zhang G,Zhang G,Prakash O,Gong YX,Soulages JL,...",True,26,3123.93


In [10]:
# DBAASP concentration calculation
concentration = list()
new_con = list()

for con in DBAASP_df['TARGET ACTIVITY - CONCENTRATION']: 
    if con.find('-') != -1:
        concentration.append(float(re.findall(r'-?\d+\.?\d*',con)[0]))
    elif con.find('±') != -1:            
        a = float(re.findall(r'-?\d+\.?\d*',con)[0])
        b = float(re.findall(r'-?\d+\.?\d*',con)[1])
        c = round(a-b,3) 
        if c < 0:
            c = a
        concentration.append(c)
    elif con.find('<') != -1:
        concentration.append(float(re.findall(r'-?\d+\.?\d*',con)[0]))
    elif con.find('>') != -1:
        concentration.append(float(re.findall(r'-?\d+\.?\d*',con)[0]))  
    else:
        concentration.append(float(re.findall(r'-?\d+\.?\d*',con)[0]))
DBAASP_df['NEW-CONCENTRATION'] = concentration
new_con = (DBAASP_df.loc[DBAASP_df['TARGET ACTIVITY - UNIT'] =='µg/ml']['NEW-CONCENTRATION']/DBAASP_df.loc\
           [DBAASP_df['TARGET ACTIVITY - UNIT'] =='µg/ml']['MW'])*1000
DBAASP_df.loc[DBAASP_df['TARGET ACTIVITY - UNIT'] =='µg/ml',['NEW-CONCENTRATION']] = new_con
DBAASP_df['log-NEW-CONCENTRATION'] = DBAASP_df['NEW-CONCENTRATION'].map(lambda x: math.log10(x) if x >0 else 0 )

DBAASP_df.head()

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,PDB NAME,...,ARTICLES - PAGES,ARTICLES - TITLE,ARTICLES - ADDITIONAL,ARTICLES - PUBMED,ARTICLES - AUTHORS,NATURE,SEQ_LEN,MW,NEW-CONCENTRATION,log-NEW-CONCENTRATION
109,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,11138-11142,"""Libraries from libraries"": chemical transform...",NaN,http://www.ncbi.nlm.nih.gov/pubmed/7972024,"Ostresh JM,Husar GM,Blondelle SE,Dörner B,Webe...",True,6,815.04,7.361602,0.866972
110,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,13.496270,1.130214
111,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,2.085787,0.319270
112,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,3.680801,0.565942
113,11,Monomer,"Cathelicidin-1, CATH-1, Fowlicidin-1",NaN,RVKRVWPLVIRTVIAGYNLYRAIKKK,NaN,Ribosomal,"Gram+,Gram-,Cancer,Mammalian Cell",Lipid Bilayer,2AMN,...,2581-2593,Structure-activity relationships of fowlicidin...,NaN,http://www.ncbi.nlm.nih.gov/pubmed/16817888,"Zhang G,Zhang G,Prakash O,Gong YX,Soulages JL,...",True,26,3123.93,2.000000,0.301030


In [10]:
#dbAMP DRAMP
def cal_concentration(con):
    if con.find('-') != -1:
        con = (float(re.findall(r'-?\d+\.?\d*',con)[0]))
    elif con.find('±') != -1:            
        a = float(re.findall(r'-?\d+\.?\d*',con)[0])
        b = float(re.findall(r'-?\d+\.?\d*',con)[1])
        con = round(a-b,3)   
    elif con.find('<') != -1:
        con = (float(re.findall(r'-?\d+\.?\d*',con)[0]))
    elif con.find('>') != -1:
        con = (float(re.findall(r'-?\d+\.?\d*',con)[0]))  
    else:
        con = (float(re.findall(r'-?\d+\.?\d*',con)[0]))

    return con

def Transform_concentration(new_df):
    new_df.loc[new_df['UNIT'] == 'μg/ml','MIC_VALUE'] = pd.to_numeric(new_df.loc[new_df['UNIT'] == 'μg/ml']['MIC_VALUE'])/new_df.loc[new_df['UNIT'] =='μg/ml']['MW']*1000
    new_df['NEW-CONCENTRATION'] = new_df['MIC_VALUE'].map(lambda x: math.log10(float(x)) if float(x) >0 else 0 )
    return new_df

In [11]:
dbAMP_df.head()

,dbAMP_ID,SEQUENCE,activities,experimental evidence,Name,Source,Tax,Uniprot,PDB,Targets,NATURE,SEQ_LEN,MW
0,dbAMP_00005,AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,"jellyfish, Aurelia aurita&&Aurelin (jellyfish,...","jellyfish, Aurelia aurita&&Aurelia aurita (Moo...","Eukaryota,Metazoa,Cnidaria,Scyphozoa,Semaeosto...",Q0MWV8&& P84891&&Q0MWV8,2LG4,Listeria monocytogenes EGD (MIC=22.64µg/ml)&&G...,True,40,4285.09
1,dbAMP_00024,AAFRGCWTKNYSPKPCL,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,"Ranacyclin-B-AL1 (Frogs, amphibians, animals)&...",Amolops loloensis (Ranidae frogs)&&Amolops lol...,"Eukaryota,Metazoa,Chordata,Amphibia,Anura,Rani...",NaN,NaN,Staphylococcus aureus ATCC 25923 (MIC=51.5µM)&...,True,17,1924.34
2,dbAMP_00027,AAGGVKKPKKAAAAKKSPKKPKKPAAA,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,VK13 (histone derived; reptiles; animals)&&VK1...,"blood plasma, Komodo dragon, Varanus komodoensis",NaN,NaN,NaN,Lactobacillus rhamnosus 7469 (MIC=>64µM)&&Lact...,True,27,2610.31
3,dbAMP_00028,AAGMGFFGAR,"['Antibacterial', 'AntiGram\xa0+', 'AntiGram&n...",YES,Urechistachykinins II&&Urechistachykinin-2&&Ur...,Urechis unicinctus,"Annelida, Urechis unicinctus &&Animalia, Annel...",P40752,NaN,[Ref.18570895] GrµM-negative bacteria: Escheri...,True,10,966.14
4,dbAMP_00030,AAGYPFGIKV,"['AntiGram\xa0+', 'AntiGram&nbsp-', 'Antifungal']",NO,Ishikawain-1,NaN,"Odorrana ishikawae&&Eukaryota,Metazoa,Chordata...",NaN,NaN,Staphylococcus aureus ATCC 6538 &&Staphylococc...,True,10,1004.21


In [13]:
dbAMP_ESKAPEE = pd.DataFrame(columns=['ID','SEQUENCE','TARGETS','SEQ_LEN','MW','MIC','MIC_VALUE','UNIT'])
regex = r'\(MIC=\S+\)'

for s in strain:
    for i in range(len(dbAMP_df)):
        res1 = [string for string in dbAMP_df['Targets'][i].split('&&') if s in string]
        if len(res1) >= 2:
            final = []
            for k in range(len(res1)):
                if 'MIC' not in res1[k]:
                    continue
                res2 = re.search(regex, res1[k])
                if res2 is None:
                    continue
                res3 = cal_concentration(res2.group(0))
                x = (res2.group(0), res3)
                final.append(x)
            if not final:
                continue
            sorted_by_second = sorted(final, key=lambda tup: tup[1])
            res2 = final[0]
            res3 = final[0][1]
        else:
            if len(res1) == 0:
                continue
            else:
                if 'MIC' not in res1[0]:
                    continue
                res2 = re.search(regex, res1[0])
                if res2 is None:
                    continue
                res3 = cal_concentration(res2.group(0))
        unit = 'µM' if 'µM' in res2[0] else 'μg/ml'
        print(res1,'\n', s, res2[0], res3, unit)
        new_row = [{'ID':dbAMP_df.loc[i].dbAMP_ID,'SEQUENCE':dbAMP_df.loc[i].SEQUENCE,'TARGETS':s,
                    'SEQ_LEN':dbAMP_df.loc[i].SEQ_LEN,'MW':dbAMP_df.loc[i].MW,'MIC':res2[0],
                    'MIC_VALUE':res3,'UNIT':unit}]
        dbAMP_ESKAPEE = dbAMP_ESKAPEE.append(new_row, ignore_index=True)
dbAMP_ESKAPEE = Transform_concentration(dbAMP_ESKAPEE)

['Staphylococcus aureus ATCC 25923 (MIC=51.5µM)', 'GrµM-positive bacteria: Staphylococcus aureus µMIC=51 µM), Bacillus subtilis µMIC=26 µM).'] 
 Staphylococcus aureus (MIC=51.5µM) 51.5 µM
['GrµM-positive bacteria: StreptococcusµMutans µMIC=12.7µM), Staphylococcus aureus µMIC=25.4µM),Enterococcus faeciµMµMIC=25.4µM) ;', 'Staphylococcus aureus ATCC 25923 (MIC=25.4µM)'] 
 Staphylococcus aureus (MIC=25.4µM) 25.4 µM
['Staphylococcus aureus MR (MIC=30µM)', 'Staphylococcus aureus KCTC 1621 (MIC=7.5µM)', 'Staphylococcus aureus ATCC 6538P (MIC=5µg/ml)', 'Staphylococcus aureus (MIC=>64µg/ml)'] 
 Staphylococcus aureus (MIC=30µM) 30.0 µM
['Staphylococcus aureus MR (MIC=16-32µg/ml)', 'Staphylococcus aureus ATCC 43300 (MIC=>128µM)', 'Staphylococcus aureus ATCC 29213 (MIC=>128µM)', 'Staphylococcus aureus ATCC 25923 (MIC=64µM)', 'Staphylococcus aureus ATCC 25923 (MBC=>128µM)'] 
 Staphylococcus aureus (MIC=16-32µg/ml) 16.0 μg/ml
['GrµM-positive bacteria: Staphylococcus aureus µMIC=5.83 ?gµMl), Bacillus

['Staphylococcus aureus ATCC 25923 (MIC=>200µg/ml)'] 
 Staphylococcus aureus (MIC=>200µg/ml) 200.0 μg/ml
['Staphylococcus aureus SH1000 (MIC=50µM)', 'Staphylococcus aureus ATCC 43300 (MIC=>20µM)'] 
 Staphylococcus aureus (MIC=50µM) 50.0 µM
['Staphylococcus aureus ATCC 25923 (MIC=25µg/ml)', 'GrµM-positive bacteria: Staphylococcus aureus ATCC25923 µMIC=25 µgµMl), Streptococcus pneµMoniae DµM11865 µMIC=37.5 µgµMl).'] 
 Staphylococcus aureus (MIC=25µg/ml) 25.0 μg/ml
['Staphylococcus aureus ATCC 6538 (MIC=12500µg/ml)', 'Staphylococcus aureus ATCC 29213 (MIC=12500µg/ml)', 'Staphylococcus aureus ATCC 25923 (MIC=12500µg/ml)'] 
 Staphylococcus aureus (MIC=12500µg/ml) 12500.0 μg/ml
['Staphylococcus aureus ATCC 25923 (MIC=200µM)'] 
 Staphylococcus aureus (MIC=200µM) 200.0 µM
['GrµM-positive bacteria: Staphylococcus aureus µMIC=4.68 ?gµMl), Bacillus subtilis µMIC=75 ?gµMl).', 'Staphylococcus aureus (MIC=4.68µg/ml)'] 
 Staphylococcus aureus (MIC=4.68µg/ml) 4.68 μg/ml
['Staphylococcus aureus (MIC=0.

['[Ref.21549739]GrµM-positive bacteria: Staphylococcus aureus ATCC 25923 µMIC=30 µM), Staphylococcus epideµMidis (clinical sµMple) µMIC=30 µM),µMicrococcus luteus ATCC 10240 µMIC>30 µM), Streptococcus pyogenes (clinical sµMple) µMIC>30 µM), Bacillus subtilis ATCC 6633 µMIC=30 µM);', 'Staphylococcus aureus ATCC 25923 (MIC=30µM)'] 
 Staphylococcus aureus (MIC=30µM) 30.0 µM
['[Ref.15246874]GrµM-positive bacteria: Staphylococcus aureus µMIC<6.25 µM), Bacillus subtilis µMIC<50 µM);', 'Staphylococcus aureus IFO 12732 (MIC=<6.25µM)'] 
 Staphylococcus aureus (MIC=<6.25µM) 6.25 µM
[' GrµM-positive bacteria: Staphylococcus aureus ATCC 25923µMIC=12.5 µM); E. faecalis ATCC 29212µMIC=50 µM); BacillusµMegateriµMµMIC=3 µM);', 'Staphylococcus aureus USA 300 (MIC=50µM)', 'Staphylococcus aureus USA 100 (MIC=50µM)', 'Staphylococcus aureus JC114 (MIC=121µM)', 'Staphylococcus aureus ATCC BAA-44 (MIC=25µM)', 'Staphylococcus aureus ATCC 33591 (MIC=4.5±1.0µM)', 'Staphylococcus aureus ATCC 33591 (MIC=1.7±0.3µM

['Staphylococcus aureus NCTC 8325 (MIC=12µM)', 'GrµM-positive bacteriµM: Staphylococcus aureus.'] 
 Staphylococcus aureus (MIC=12µM) 12.0 µM
['GrµM-positive bacteria: Staphylococcus aureus µMIC=12.5 ?gµMl), Bacillus subtilis µMIC=50 ?gµMl);', 'Staphylococcus aureus ATCC 2592 (MIC=12.5µg/ml)'] 
 Staphylococcus aureus (MIC=12.5µg/ml) 12.5 μg/ml
['Staphylococcus aureus ATCC 2592 (MIC=6.25µg/ml)'] 
 Staphylococcus aureus (MIC=6.25µg/ml) 6.25 μg/ml
['Staphylococcus aureus (MIC=130µM)', 'GrµM-positive bacteriµM: Staphylococcus aureus µMIC=130 µM).'] 
 Staphylococcus aureus (MIC=130µM) 130.0 µM
['Staphylococcus aureus (MIC=100µM)', 'GrµM-positive bacteriµM: Staphylococcus aureus µMIC=100 µM).'] 
 Staphylococcus aureus (MIC=100µM) 100.0 µM
['Staphylococcus aureus ATCC 25923 (MIC=25µM)'] 
 Staphylococcus aureus (MIC=25µM) 25.0 µM
['Staphylococcus aureus ATCC 25923 (MIC=100µM)'] 
 Staphylococcus aureus (MIC=100µM) 100.0 µM
['Staphylococcus aureus ATCC 25923 (MIC=12.5µM)'] 
 Staphylococcus aureus

['Staphylococcus aureus PTCC 1431 (MIC=35.2±1µg/ml)', 'GrµM-positive bacteria: Streptococcus dysgalactiae PTCC 1074 µMIC=33.2 µgµMl), Streptococcus agalactiae PTCC 1365 µMIC=24.7 µgµMl) and Staphylococcus aureus PTCC 1431 µMIC=35.2 µgµMl).'] 
 Staphylococcus aureus (MIC=35.2±1µg/ml) 34.2 μg/ml
['Staphylococcus aureus ATCC 2592 (MIC=37.5µg/ml)'] 
 Staphylococcus aureus (MIC=37.5µg/ml) 37.5 μg/ml
['GrµM-positive bacteria:µMicrococcus luteus PTCC1625 µMIC=4.7 µM), Staphylococcus aureus PTCC1431 µMIC=5.3 µM).', 'Staphylococcus aureus PTCC 1431 (MIC=5.3µM)'] 
 Staphylococcus aureus (MIC=5.3µM) 5.3 µM
['Staphylococcus aureus IID 1677 (MIC=20.0µg/ml)', 'Staphylococcus aureus C623 (MIC=17µg/ml)', 'Staphylococcus aureus ATCC 25923 (MIC=40µg/ml)'] 
 Staphylococcus aureus (MIC=20.0µg/ml) 20.0 μg/ml
['Staphylococcus aureus NCTC 10788 (MIC=>200µM)'] 
 Staphylococcus aureus (MIC=>200µM) 200.0 µM
['[Ref.10651828]GrµM-positive bacteriµM: Staphylococcus aureus µMIC=1 µM);', 'Staphylococcus aureus NCTC 

...

 Escherichia coli (MIC=4µM) 4.0 µM
['Escherichia coli WBB 01 (MIC=1.25µM)', 'Escherichia coli WBB 01 (MIC=0.3µM)', 'Escherichia coli WBB 01 (MBC=2.5µM)', 'Escherichia coli WBB 01 (MBC=0.6µM)', 'Escherichia coli ML-35p (MIC=4µg/ml)'] 
 Escherichia coli (MIC=1.25µM) 1.25 µM
['Escherichia coli WBB 01 (MIC=2.5µM)', 'Escherichia coli WBB 01 (MBC=5µM)'] 
 Escherichia coli (MIC=2.5µM) 2.5 µM
['Escherichia coli WBB 01 (MIC=2.5µM)', 'Escherichia coli WBB 01 (MBC=5µM)'] 
 Escherichia coli (MIC=2.5µM) 2.5 µM
['Escherichia coli WBB 01 (MIC=1.25µM)', 'Escherichia coli WBB 01 (MBC=2.5µM)'] 
 Escherichia coli (MIC=1.25µM) 1.25 µM
['Escherichia coli WBB 01 (MIC=1.25µM)', 'Escherichia coli WBB 01 (MBC=5µM)'] 
 Escherichia coli (MIC=1.25µM) 1.25 µM
['Escherichia coli WBB 01 (MIC=1.25µM)', 'Escherichia coli WBB 01 (MBC=5µM)'] 
 Escherichia coli (MIC=1.25µM) 1.25 µM
['Escherichia coli ML-35p (MIC=1-2µM)', 'Escherichia coli C600 (MIC=1.56µM)', 'Escherichia coli ATCC 25922 (MIC=2µM)', 'Escherichia coli ATCC

['Escherichia coli KCTC 1682 (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=20µg/ml)'] 
 Escherichia coli (MIC=20µg/ml) 20.0 μg/ml
['Escherichia coli UB1005 (MIC=64µg/ml)', 'Escherichia coli UB1005 (MIC=32µg/ml)', 'Escherichia coli (MIC=>100µg/ml)'] 
 Escherichia coli (MIC=64µg/ml) 64.0 μg/ml
['Escherichia coli NIHJ JC-2 (MIC=4µg/ml)', 'Escherichia coli NIHJ JC-2 (MIC=34µg/ml)'] 
 Escherichia coli (MIC=4µg/ml) 4.0 μg/ml
['Escherichia coli KCTC 1682 (MIC=20µg/ml)', 'Escherichia coli CCARM 1238 (MIC=2µM)', 'Escherichia coli CCARM 1229 (MIC=4µM)'] 
 Escherichia coli (MIC=20µg/ml) 20.0 μg/ml
['Escherichia coli KCTC 1682 (MIC=10µg/ml)'] 
 Escherichia coli (MIC=10µg/ml) 10.0 μg/ml
['Escherichia coli KCTC 1682 (MIC=32µg/ml)'] 
 Escherichia coli (MIC=32µg/ml) 32.0 μg/ml
['Escherichia coli ATCC 25922 (MIC=>160µM)'] 
 Escherichia coli (MIC=>160µM) 160.0 µM
['Escherichia coli O157 (MIC=>128µg/ml)', 'Escherichia coli CVCC 195 (MIC=>128µg/ml)'] 
 Escherichia coli

 Escherichia coli (MIC=>128µM) 128.0 µM
['Escherichia coli ATCC 25922 (MIC=>128µM)'] 
 Escherichia coli (MIC=>128µM) 128.0 µM
['Escherichia coli ATCC 25922 (MIC=128µM)'] 
 Escherichia coli (MIC=128µM) 128.0 µM
['Escherichia coli ATCC 25922 (MIC=>128µM)'] 
 Escherichia coli (MIC=>128µM) 128.0 µM
['Escherichia coli ATCC 25922 (MIC=4µM)'] 
 Escherichia coli (MIC=4µM) 4.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)', 'Escherichia coli ATCC 25922 (MIC=32µM)', 'Escherichia coli ATCC 25922 (MIC=16µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)', 'Escherichia coli (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli ATCC 25922 (MIC=8µM)'] 
 

['Escherichia coli KCTC 1682 (MIC=2µM)', 'Escherichia coli CCARM 1238 (MIC=4µM)', 'Escherichia coli CCARM 1229 (MIC=8µM)'] 
 Escherichia coli (MIC=2µM) 2.0 µM
['Escherichia coli ML-35 (MIC=8µM)', 'Escherichia coli ML-35 (MIC=2µM)'] 
 Escherichia coli (MIC=8µM) 8.0 µM
['Escherichia coli K99 (MIC=2µg/ml)', 'Escherichia coli K91BK (MIC=100µM)', 'Escherichia coli K91BK (IC50=70.7µM)', 'Escherichia coli K88 (MIC=2µg/ml)', 'Escherichia coli K-12 D31 (MIC=1.8µg/ml)', 'Escherichia coli ATCC 25922 (MIC=6.3µg/ml)'] 
 Escherichia coli (MIC=2µg/ml) 2.0 μg/ml
['Escherichia coli LWF+ (MIC=25µM)', 'Escherichia coli LWF+ (MIC=19µM)', 'Escherichia coli DH5alpha (MIC=9µM)', 'Escherichia coli DH5alpha (MIC=11µM)', 'Escherichia coli ATCC 25922 (MIC=4µg/ml)'] 
 Escherichia coli (MIC=25µM) 25.0 µM
['Escherichia coli K91BK (MIC=100µM)', 'Escherichia coli K91BK (IC50=70.7µM)'] 
 Escherichia coli (MIC=100µM) 100.0 µM
['Escherichia coli K91BK (MIC=>100µM)', 'Escherichia coli K91BK (IC50=>100µM)'] 
 Escherichia 

['Escherichia coli ATCC 25726 (MIC=6µM)'] 
 Escherichia coli (MIC=6µM) 6.0 µM
['Escherichia coli ATCC 25726 (MIC=25µM)', 'Escherichia coli ATCC 25726 (MIC=12µM)'] 
 Escherichia coli (MIC=25µM) 25.0 µM
['Escherichia coli ATCC 25726 (MIC=6µM)', 'Escherichia coli ATCC 25726 (MIC=3µM)'] 
 Escherichia coli (MIC=6µM) 6.0 µM
['Escherichia coli (MIC=12.50µM)', 'Escherichia coli (MIC=11.25µM)'] 
 Escherichia coli (MIC=12.50µM) 12.5 µM
['Escherichia coli ATCC 25629 (MIC=12.5µg/ml)', 'Escherichia coli (MIC=15.0µg/ml)'] 
 Escherichia coli (MIC=12.5µg/ml) 12.5 μg/ml
['Escherichia coli ATCC 25726 (MIC=20µM)', 'Escherichia coli ATCC 25726 (MIC=10µM)'] 
 Escherichia coli (MIC=20µM) 20.0 µM
['Escherichia coli Ec513 (MIC=32µg/ml)', 'Escherichia coli ATCC 25726 (MIC=80µM)', 'Escherichia coli ATCC 25726 (MIC=5µM)'] 
 Escherichia coli (MIC=32µg/ml) 32.0 μg/ml
['Escherichia coli ATCC 25726 (MIC=10µM)'] 
 Escherichia coli (MIC=10µM) 10.0 µM
['Escherichia coli ATCC 25726 (MIC=5µM)', 'Escherichia coli ATCC 257

In [14]:
dbAMP_ESKAPEE

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION
0,dbAMP_00024,AAFRGCWTKNYSPKPCL,Staphylococcus aureus,17,1924.34,(MIC=51.5µM),51.500000,µM,1.711807
1,dbAMP_00028,AAGMGFFGAR,Staphylococcus aureus,10,966.14,(MIC=25.4µM),25.400000,µM,1.404834
2,dbAMP_00032,AAHCLAIGRR,Staphylococcus aureus,10,1049.28,(MIC=30µM),30.000000,µM,1.477121
3,dbAMP_00037,AAKIILNPKFRCKAAFC,Staphylococcus aureus,17,1876.42,(MIC=16-32µg/ml),8.526876,μg/ml,0.930790
4,dbAMP_00046,AALKGCWTKSIPPKPCFGKR,Staphylococcus aureus,20,2170.74,(MIC=5.83µg/ml),2.685720,μg/ml,0.429061
...,...,...,...,...,...,...,...,...,...
15869,dbAMP_28519,RRWQWRMKKLG,Escherichia coli,11,1526.91,(MIC=64µg/ml),41.914717,μg/ml,1.622367
15870,dbAMP_28628,VRRFPAAAPFLRR,Escherichia coli,13,1538.89,(MIC=>64µg/ml),41.588418,μg/ml,1.618972
15871,dbAMP_28635,VTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR,Escherichia coli,35,3769.38,(MIC=87µM),87.000000,µM,1.939519
15872,dbAMP_28639,WKKIRVRLSA,Escherichia coli,10,1238.57,(MIC=>100µM),100.000000,µM,2.000000


In [15]:
DRAMP_ESKAPEE = pd.DataFrame(columns=['ID','SEQUENCE','TARGETS','SEQ_LEN','MW','MIC','MIC_VALUE','UNIT'])
regex = r'\(MIC=\S+\)'
for s in strain:
    for i in range(len(DRAMP_df)):
        res1 = [string for string in re.split(',| ;|##', DRAMP_df['Target_Organism'][i]) if s in string]
        if len(res1)==0:
            continue
        if len(res1) >= 2:
            final = []
            for k in range(len(res1)):
                if 'MIC' not in res1[k]:
                    continue
                res2 = re.search(regex, res1[k])
                if res2 is None:
                    continue
                res3 = cal_concentration(res2.group(0))
                x = (res2.group(0), res3)
                final.append(x)
            if not final:
                continue
            sorted_by_second = sorted(final, key=lambda tup: tup[1])
            res2 = final[0]
            res3 = final[0][1]
        else:
            if len(res1) == 0:
                continue
            else:
                if 'MIC' not in res1[0]:
                    continue
                res2 = re.search(regex, res1[0])
                if res2 is None:
                    continue
                res3 = cal_concentration(res2.group(0))
        unit = 'μM' if 'μM' in res2[0] else ('μg/ml' if ('μg/ml' in res2[0]) or ('μg/mL' in res2[0]) else 'NONE')
#         print(res1,'\n',s,i, res2[0], res3, unit)
        new_row = [{'ID':DRAMP_df.loc[i].DRAMP_ID,'SEQUENCE':DRAMP_df.loc[i].SEQUENCE,'TARGETS':s,
                    'SEQ_LEN':DRAMP_df.loc[i].SEQ_LEN,'MW':DRAMP_df.loc[i].MW,'MIC':res2[0],
                    'MIC_VALUE':res3,'UNIT':unit}]
        DRAMP_ESKAPEE = DRAMP_ESKAPEE.append(new_row, ignore_index=True)

DRAMP_ESKAPEE = DRAMP_ESKAPEE[DRAMP_ESKAPEE['UNIT'] != 'NONE']
DRAMP_ESKAPEE = Transform_concentration(DRAMP_ESKAPEE)

In [16]:
DRAMP_ESKAPEE

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION
0,DRAMP03017,FMPIIGRLMSGSL,Staphylococcus aureus,13,1403.79,(MIC=1.9μg/ml),1.353479,μg/ml,0.131451
1,DRAMP03722,IFGAIWNGIKSLF,Staphylococcus aureus,13,1447.77,(MIC=50μg/mL),34.535872,μg/ml,1.538270
2,DRAMP04213,KKLFKKILKYL,Staphylococcus aureus,11,1403.90,(MIC=2μM),2.000000,μM,0.301030
3,DRAMP03028,IDWKKLLDAAKQIL,Staphylococcus aureus,14,1637.03,(MIC=18.8μM),18.800000,μM,1.274158
4,DRAMP03721,ILGKIWEGIKSLF,Staphylococcus aureus,13,1485.86,(MIC=50μg/mL),33.650546,μg/ml,1.526992
...,...,...,...,...,...,...,...,...,...
301,DRAMP21463,IHKFWRRWFKHI,Escherichia coli,12,1736.14,(MIC=16μM),16.000000,μM,1.204120
302,DRAMP21464,IHFKWRRWKFHI,Escherichia coli,12,1736.14,(MIC=32μM),32.000000,μM,1.505150
303,DRAMP21465,DEMKLDGFNMHLE,Escherichia coli,13,1560.80,(MIC=20μM),20.000000,μM,1.301030
304,DRAMP21466,AAGKGLVSNLLEK,Escherichia coli,13,1281.54,(MIC=24.6μM),24.600000,μM,1.390935


In [17]:
DBAASP_df.head()

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT,PDB NAME,...,ARTICLES - PAGES,ARTICLES - TITLE,ARTICLES - ADDITIONAL,ARTICLES - PUBMED,ARTICLES - AUTHORS,NATURE,SEQ_LEN,MW,NEW-CONCENTRATION,log-NEW-CONCENTRATION
109,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,11138-11142,"""Libraries from libraries"": chemical transform...",NaN,http://www.ncbi.nlm.nih.gov/pubmed/7972024,"Ostresh JM,Husar GM,Blondelle SE,Dörner B,Webe...",True,6,815.04,7.361602,0.866972
110,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,13.496270,1.130214
111,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,2.085787,0.319270
112,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,Gram+,Lipid Bilayer,NaN,...,NaN,NaN,NaN,NaN,NaN,True,6,815.04,3.680801,0.565942
113,11,Monomer,"Cathelicidin-1, CATH-1, Fowlicidin-1",NaN,RVKRVWPLVIRTVIAGYNLYRAIKKK,NaN,Ribosomal,"Gram+,Gram-,Cancer,Mammalian Cell",Lipid Bilayer,2AMN,...,2581-2593,Structure-activity relationships of fowlicidin...,NaN,http://www.ncbi.nlm.nih.gov/pubmed/16817888,"Zhang G,Zhang G,Prakash O,Gong YX,Soulages JL,...",True,26,3123.93,2.000000,0.301030


In [18]:
DBAASP_ESKAPEE = pd.DataFrame()
for s in strain:
    new_row = DBAASP_df[DBAASP_df['TARGET ACTIVITY - TARGET SPECIES'].str.contains(s)==True]
    DBAASP_ESKAPEE = DBAASP_ESKAPEE.append(new_row,ignore_index=True)
DBAASP_ESKAPEE['ID'] = DBAASP_ESKAPEE['ID'].map(lambda x: f'DBAASP_{x}')
DBAASP_ESKAPEE = DBAASP_ESKAPEE[DBAASP_ESKAPEE['TARGET ACTIVITY - ACTIVITY MEASURE GROUP']=='MIC']
DBAASP_ESKAPEE = DBAASP_ESKAPEE[['ID','SEQUENCE','TARGET ACTIVITY - TARGET SPECIES','SEQ_LEN','MW','TARGET ACTIVITY - CONCENTRATION','NEW-CONCENTRATION',
                'TARGET ACTIVITY - UNIT','log-NEW-CONCENTRATION']]
DBAASP_ESKAPEE = DBAASP_ESKAPEE.rename({'TARGET ACTIVITY - TARGET SPECIES': 'TARGETS',
                                       'TARGET ACTIVITY - CONCENTRATION':'MIC',
                                       'NEW-CONCENTRATION':'MIC_VALUE',
                                       'TARGET ACTIVITY - UNIT':'UNIT',
                                       'log-NEW-CONCENTRATION':'NEW-CONCENTRATION'}, axis=1)
DBAASP_ESKAPEE

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION
1,DBAASP_10,LFIFFF,Staphylococcus aureus,6,815.04,11-15,13.496270,µg/ml,1.130214
2,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 25923,26,3123.93,0.80,0.800000,µM,-0.096910
3,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.40,0.400000,µM,-0.397940
4,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.80,0.800000,µM,-0.096910
5,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 43300,26,3123.93,0.40,0.400000,µM,-0.397940
...,...,...,...,...,...,...,...,...,...
46485,DBAASP_20296,WNSNVHSYRF,Escherichia coli MG1655,10,1291.42,16,12.389463,µg/ml,1.093052
46486,DBAASP_20296,WNSNVHSYRF,Escherichia coli MG1655 bamA,10,1291.42,>128,99.115702,µg/ml,1.996142
46487,DBAASP_20296,WNSNVHSYRF,Escherichia coli BW25113,10,1291.42,8,6.194731,µg/ml,0.792022
46488,DBAASP_20296,WNSNVHSYRF,Escherichia coli BW25113 del-ompF,10,1291.42,8,6.194731,µg/ml,0.792022


In [19]:
print(len(DRAMP_ESKAPEE),len(dbAMP_ESKAPEE),len(DBAASP_ESKAPEE))

256 15874 36406


In [20]:
DRAMP_ESKAPEE.head()

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION
0,DRAMP03017,FMPIIGRLMSGSL,Staphylococcus aureus,13,1403.79,(MIC=1.9μg/ml),1.353479,μg/ml,0.131451
1,DRAMP03722,IFGAIWNGIKSLF,Staphylococcus aureus,13,1447.77,(MIC=50μg/mL),34.535872,μg/ml,1.538270
2,DRAMP04213,KKLFKKILKYL,Staphylococcus aureus,11,1403.90,(MIC=2μM),2.000000,μM,0.301030
3,DRAMP03028,IDWKKLLDAAKQIL,Staphylococcus aureus,14,1637.03,(MIC=18.8μM),18.800000,μM,1.274158
4,DRAMP03721,ILGKIWEGIKSLF,Staphylococcus aureus,13,1485.86,(MIC=50μg/mL),33.650546,μg/ml,1.526992


In [36]:
# DBAASP_ESKAPEE['TARGETS'].value_counts()
DBAASP_ESKAPEE[DBAASP_ESKAPEE['TARGETS'].str.contains('Pseudomonas aeruginosa') == True].TARGETS.value_counts()[:30]


Pseudomonas aeruginosa ATCC 27853      2750
Pseudomonas aeruginosa PAO1             936
Pseudomonas aeruginosa                  894
Pseudomonas aeruginosa ATCC 9027        536
Pseudomonas aeruginosa KCTC 1637        319
Pseudomonas aeruginosa ATCC 15692       123
Pseudomonas aeruginosa H103             114
Pseudomonas aeruginosa CMCCB 10104      103
Pseudomonas aeruginosa PA14              93
Pseudomonas aeruginosa Ps4               90
Pseudomonas aeruginosa K799              85
Pseudomonas aeruginosa ATCC 15442        85
Pseudomonas aeruginosa ATCC 10145        81
Pseudomonas aeruginosa CGMCC 1.860       73
Pseudomonas aeruginosa CCARM 2095        67
Pseudomonas aeruginosa H1001             66
Pseudomonas aeruginosa Z61               60
Pseudomonas aeruginosa PAK               55
Pseudomonas aeruginosa ATCC BAA-427      53
Pseudomonas aeruginosa CICC 21630        51
Pseudomonas aeruginosa 50                41
Pseudomonas aeruginosa DSM 1117          41
Pseudomonas aeruginosa ATCC 1338

In [27]:
DBAASP_dbAMP_DRAMP = pd.concat([DBAASP_ESKAPEE, dbAMP_ESKAPEE, DRAMP_ESKAPEE], axis=0,ignore_index=True)
DBAASP_dbAMP_DRAMP

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION
0,DBAASP_10,LFIFFF,Staphylococcus aureus,6,815.04,11-15,13.49627,µg/ml,1.130214
1,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 25923,26,3123.93,0.80,0.80000,µM,-0.096910
2,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.40,0.40000,µM,-0.397940
3,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.80,0.80000,µM,-0.096910
4,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 43300,26,3123.93,0.40,0.40000,µM,-0.397940
...,...,...,...,...,...,...,...,...,...
52531,DRAMP21463,IHKFWRRWFKHI,Escherichia coli,12,1736.14,(MIC=16μM),16.00000,μM,1.204120
52532,DRAMP21464,IHFKWRRWKFHI,Escherichia coli,12,1736.14,(MIC=32μM),32.00000,μM,1.505150
52533,DRAMP21465,DEMKLDGFNMHLE,Escherichia coli,13,1560.80,(MIC=20μM),20.00000,μM,1.301030
52534,DRAMP21466,AAGKGLVSNLLEK,Escherichia coli,13,1281.54,(MIC=24.6μM),24.60000,μM,1.390935


In [28]:
with open('DBAASP_dbAMP_DRAMP.fasta', 'w') as fw:
    for _id, _seq in zip(DBAASP_dbAMP_DRAMP['ID'], DBAASP_dbAMP_DRAMP['SEQUENCE']):
        fw.write(f'>{_id}\n{_seq}\n')

In [29]:
os.system('python iFeature-master/iFeature.py --file DBAASP_dbAMP_DRAMP.fasta --out DBAASP_dbAMP_DRAMP_AAC.tsv --type AAC')
os.system('python iFeature-master/iFeature.py --file DBAASP_dbAMP_DRAMP.fasta --out DBAASP_dbAMP_DRAMP_CTDD.tsv --type CTDD')
os.system('python iFeature-master/codes/PAAC.py DBAASP_dbAMP_DRAMP.fasta 5 DBAASP_dbAMP_DRAMP_PAAC.tsv')
os.system('python iFeature-master/iFeature.py --file DBAASP_dbAMP_DRAMP.fasta --out DBAASP_dbAMP_DRAMP_GAAC.tsv --type GAAC')

0

In [235]:
def Merge_feature(dataname,dataframe):
    #Using this four features to build ML model
    temp_df = dataframe.rename({'ID': '#'}, axis=1)
    AAC = pd.read_csv('{}_AAC.tsv'.format(dataname), sep='\t')
    PAAC = pd.read_csv('{}_PAAC.tsv'.format(dataname), sep='\t')
    CTDD = pd.read_csv('{}_CTDD.tsv'.format(dataname), sep='\t')
    GAAC = pd.read_csv('{}_GAAC.tsv'.format(dataname), sep='\t')

    temp_df = pd.merge(temp_df, AAC, on='#')
    temp_df = pd.merge(temp_df, PAAC, on='#')
    temp_df = pd.merge(temp_df, CTDD, on='#')
    df = pd.merge(temp_df, GAAC, on='#')
    df = df.rename({'#': 'ID'}, axis=1)

    return df

In [30]:
AAC = pd.read_csv('DBAASP_dbAMP_DRAMP_AAC.tsv', sep='\t')
PAAC = pd.read_csv('DBAASP_dbAMP_DRAMP_PAAC.tsv', sep='\t')
CTDD = pd.read_csv('DBAASP_dbAMP_DRAMP_CTDD.tsv', sep='\t')
GAAC = pd.read_csv('DBAASP_dbAMP_DRAMP_GAAC.tsv', sep='\t')

DBAASP_dbAMP_DRAMP_features = pd.concat([DBAASP_dbAMP_DRAMP, AAC, PAAC, CTDD, GAAC], axis=1)
DBAASP_dbAMP_DRAMP_features = DBAASP_dbAMP_DRAMP_features.drop('#', axis=1)
DBAASP_dbAMP_DRAMP_features.head()

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION,A,...,solventaccess.3.residue0,solventaccess.3.residue25,solventaccess.3.residue50,solventaccess.3.residue75,solventaccess.3.residue100,alphatic,aromatic,postivecharge,negativecharge,uncharge
0,DBAASP_10,LFIFFF,Staphylococcus aureus,6,815.04,11-15,13.49627,µg/ml,1.130214,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.0,0.000000
1,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 25923,26,3123.93,0.80,0.80000,µM,-0.096910,0.076923,...,26.923077,26.923077,46.153846,65.384615,76.923077,0.461538,0.115385,0.307692,0.0,0.115385
2,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.40,0.40000,µM,-0.397940,0.076923,...,26.923077,26.923077,46.153846,65.384615,76.923077,0.461538,0.115385,0.307692,0.0,0.115385
3,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.80,0.80000,µM,-0.096910,0.076923,...,26.923077,26.923077,46.153846,65.384615,76.923077,0.461538,0.115385,0.307692,0.0,0.115385
4,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 43300,26,3123.93,0.40,0.40000,µM,-0.397940,0.076923,...,26.923077,26.923077,46.153846,65.384615,76.923077,0.461538,0.115385,0.307692,0.0,0.115385


In [262]:
# Don't call again
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Acinetobacter baumannii strain K09-14 chromosome.fasta" -o AB_NAC.csv -l DNA -t NAC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Acinetobacter baumannii strain K09-14 chromosome.fasta" -o AB_DNC.csv -l DNA -t DNC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Acinetobacter baumannii strain K09-14 chromosome.fasta" -o AB_TNC.csv -l DNA -t TNC -seq 1')

# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterobacter cloacae.fasta" -o ENC_NAC.csv -l DNA -t NAC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterobacter cloacae.fasta" -o ENC_DNC.csv -l DNA -t DNC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterobacter cloacae.fasta" -o ENC_TNC.csv -l DNA -t TNC -seq 1')

# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterococcus faecium strain SRR24 chromosome.fasta" -o EF_NAC.csv -l DNA -t NAC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterococcus faecium strain SRR24 chromosome.fasta" -o EF_DNC.csv -l DNA -t DNC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Enterococcus faecium strain SRR24 chromosome.fasta" -o EF_TNC.csv -l DNA -t TNC -seq 1')

# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Klebsiella pneumoniae subsp. pneumoniae HS11286.fasta" -o KP_NAC.csv -l DNA -t NAC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Klebsiella pneumoniae subsp. pneumoniae HS11286.fasta" -o KP_DNC.csv -l DNA -t DNC -seq 1')
# os.system('python MathFeature-master/methods/ExtractionTechniques.py -i reference_genome/"Klebsiella pneumoniae subsp. pneumoniae HS11286.fasta" -o KP_TNC.csv -l DNA -t TNC -seq 1')

0

In [12]:
# Acinetobacter baumannii
AB_NAC = pd.read_csv('reference_genome/AB_NAC.csv')
AB_DNC = pd.read_csv('reference_genome/AB_DNC.csv')
AB_TNC = pd.read_csv('reference_genome/AB_TNC.csv')
AB_NAC.drop(['label','nameseq'],axis=1,inplace=True)
AB_DNC.drop(['label','nameseq'],axis=1,inplace=True)
AB_TNC.drop(['label','nameseq'],axis=1,inplace=True)
AB_genome = pd.concat([AB_NAC,AB_DNC,AB_TNC],axis=1,join='inner')
AB_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
AB_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.305454,0.195255,0.194721,0.30457,0.110642,0.05411,0.054517,0.086185,0.069206,0.037608,...,0.009073,0.014331,0.020481,0.017631,0.015481,0.015357,0.027239,0.018477,0.024236,0.03999


In [275]:
# Enterobacter cloacae
ENC_NAC = pd.read_csv('reference_genome/ENC_NAC.csv')
ENC_DNC = pd.read_csv('reference_genome/ENC_DNC.csv')
ENC_TNC = pd.read_csv('reference_genome/ENC_TNC.csv')
ENC_NAC.drop(['label','nameseq'],axis=1,inplace=True)
ENC_DNC.drop(['label','nameseq'],axis=1,inplace=True)
ENC_TNC.drop(['label','nameseq'],axis=1,inplace=True)
ENC_genome = pd.concat([ENC_NAC,ENC_DNC,ENC_TNC],axis=1,join='inner')
ENC_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
ENC_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.224998,0.274516,0.275283,0.225203,0.060757,0.054291,0.053236,0.056714,0.06865,0.067765,...,0.01566,0.012248,0.017867,0.021102,0.019614,0.011232,0.011124,0.016506,0.01346,0.01907


In [276]:
# Enterococcus faecium
EF_NAC = pd.read_csv('reference_genome/EF_NAC.csv')
EF_DNC = pd.read_csv('reference_genome/EF_DNC.csv')
EF_TNC = pd.read_csv('reference_genome/EF_TNC.csv')
EF_NAC.drop(['label','nameseq'],axis=1,inplace=True)
EF_DNC.drop(['label','nameseq'],axis=1,inplace=True)
EF_TNC.drop(['label','nameseq'],axis=1,inplace=True)
EF_genome = pd.concat([EF_NAC,EF_DNC,EF_TNC],axis=1,join='inner')
EF_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
EF_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.308323,0.19056,0.189629,0.311488,0.113039,0.048568,0.055722,0.090995,0.064145,0.034716,...,0.011986,0.019892,0.021484,0.013681,0.013346,0.016288,0.019705,0.025635,0.02383,0.045474


In [300]:
# Klebsiella pneumoniae
KP_NAC = pd.read_csv('reference_genome/KP_NAC.csv')
KP_DNC = pd.read_csv('reference_genome/KP_DNC.csv')
KP_TNC = pd.read_csv('reference_genome/KP_TNC.csv')
KP_NAC.drop(['label','nameseq'],axis=1,inplace=True)
KP_DNC.drop(['label','nameseq'],axis=1,inplace=True)
KP_TNC.drop(['label','nameseq'],axis=1,inplace=True)
KP_genome = pd.concat([KP_NAC,KP_DNC,KP_TNC],axis=1,join='inner')
KP_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
KP_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.212908,0.287281,0.287567,0.212244,0.053721,0.04848,0.054628,0.056079,0.065009,0.073934,...,0.018563,0.010821,0.015802,0.020091,0.019148,0.009244,0.010062,0.015127,0.011578,0.01705


In [31]:
# Staphylococcus aureus
SA_NAC = pd.read_csv('reference_genome/SA_NAC.csv')
SA_DNC = pd.read_csv('reference_genome/SA_DNC.csv')
SA_TNC = pd.read_csv('reference_genome/SA_TNC.csv')
SA_NAC.drop(['label','nameseq'],axis=1,inplace=True)
SA_DNC.drop(['label','nameseq'],axis=1,inplace=True)
SA_TNC.drop(['label','nameseq'],axis=1,inplace=True)
SA_TNC.drop(['Unnamed: 66','Unnamed: 67'],axis=1,inplace=True)
SA_genome = pd.concat([SA_NAC,SA_DNC,SA_TNC],axis=1,join='inner')
SA_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
SA_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.332716,0.165109,0.163574,0.338601,0.120891,0.052089,0.04745,0.112287,0.064338,0.025963,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.03532,0.020256,0.02427,0.044469


In [32]:
# Escherichia coli 
EC_NAC = pd.read_csv('reference_genome/EC_NAC.csv')
EC_DNC = pd.read_csv('reference_genome/EC_DNC.csv')
EC_TNC = pd.read_csv('reference_genome/EC_TNC.csv')
EC_NAC.drop(['label','nameseq'],axis=1,inplace=True)
EC_DNC.drop(['label','nameseq'],axis=1,inplace=True)
EC_TNC.drop(['label','nameseq'],axis=1,inplace=True)
EC_genome = pd.concat([EC_NAC,EC_DNC,EC_TNC],axis=1,join='inner')
EC_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
EC_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.246193,0.254239,0.253668,0.2459,0.07282,0.055319,0.051278,0.066776,0.070089,0.058561,...,0.01546,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669


In [33]:
# Pseudomonas aeruginosa
PA_NAC = pd.read_csv('reference_genome/PA_NAC.csv')
PA_DNC = pd.read_csv('reference_genome/PA_DNC.csv')
PA_TNC = pd.read_csv('reference_genome/PA_TNC.csv')
PA_NAC.drop(['label','nameseq'],axis=1,inplace=True)
PA_DNC.drop(['label','nameseq'],axis=1,inplace=True)
PA_TNC.drop(['label','nameseq'],axis=1,inplace=True)
PA_genome = pd.concat([PA_NAC,PA_DNC,PA_TNC],axis=1,join='inner')
PA_genome.rename(columns={'A':'Genome_A','C':'Genome_C','G':'Genome_G','T':'Genome_T'},inplace=True)
PA_genome

,Genome_A,Genome_C,Genome_G,Genome_T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.168593,0.335656,0.329901,0.16585,0.030431,0.049002,0.056499,0.03266,0.062232,0.095128,...,0.02742,0.007856,0.010259,0.021479,0.020629,0.007288,0.00128,0.014552,0.009435,0.004421


In [317]:
temp_df.to_csv('DBAASP_dbAMP_DRAMP_features.csv',index=False)

In [34]:
# Create a dictionary mapping target organisms to genome dataframes
genome_mapping = {
    'Escherichia coli': EC_genome,
    'Staphylococcus aureus': SA_genome,
    'Pseudomonas aeruginosa': PA_genome,
#     'Klebsiella pneumoniae': KP_genome,
#     'Acinetobacter baumannii': AB_genome,
#     'Enterococcus faecium': EF_genome,
#     'Enterobacter cloacae': ENC_genome
}

# Create an empty dataframe to store the results
temp_df = pd.DataFrame()

# Loop through DBAASP_dbAMP_DRAMP_features
for i in range(len(DBAASP_dbAMP_DRAMP_features)):
    targets = DBAASP_dbAMP_DRAMP_features.loc[i].TARGETS
    for target in genome_mapping:
        if target in targets:
            res = DBAASP_dbAMP_DRAMP_features.loc[[i]].join(genome_mapping[target], how='cross')
            temp_df = temp_df.append(res)
            break

In [36]:
temp_df.to_csv('DBAASP_dbAMP_DRAMP_features_1.csv',index=False)

In [48]:
Final_df = pd.DataFrame()
def Remove_all_same_sequence(dataframe):
    _dataframe = pd.DataFrame()
    _seq = set(dataframe['SEQUENCE'])
    for SEQUENCE in _seq:
        temp = dataframe[dataframe['SEQUENCE'] == SEQUENCE]
        temp = temp[temp['NEW-CONCENTRATION'] == temp['NEW-CONCENTRATION'].min()]
        temp = temp.drop_duplicates(subset=['SEQUENCE'], keep='first')
        _dataframe = _dataframe.append(temp,ignore_index=True)
    return _dataframe
for s in strain:
    res = temp_df[temp_df.TARGETS.str.contains(s)==True]
    res = Remove_all_same_sequence(res)
    Final_df = Final_df.append(res)

In [50]:
temp_df

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION,A,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,DBAASP_10,LFIFFF,Staphylococcus aureus,6,815.04,11-15,13.49627,µg/ml,1.130214,0.000000,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
0,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 25923,26,3123.93,0.80,0.80000,µM,-0.096910,0.076923,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
0,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.40,0.40000,µM,-0.397940,0.076923,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
0,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC BAA-39,26,3123.93,0.80,0.80000,µM,-0.096910,0.076923,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
0,DBAASP_11,RVKRVWPLVIRTVIAGYNLYRAIKKK,Staphylococcus aureus ATCC 43300,26,3123.93,0.40,0.40000,µM,-0.397940,0.076923,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DRAMP21463,IHKFWRRWFKHI,Escherichia coli,12,1736.14,(MIC=16μM),16.00000,μM,1.204120,0.000000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
0,DRAMP21464,IHFKWRRWKFHI,Escherichia coli,12,1736.14,(MIC=32μM),32.00000,μM,1.505150,0.000000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
0,DRAMP21465,DEMKLDGFNMHLE,Escherichia coli,13,1560.80,(MIC=20μM),20.00000,μM,1.301030,0.000000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
0,DRAMP21466,AAGKGLVSNLLEK,Escherichia coli,13,1281.54,(MIC=24.6μM),24.60000,μM,1.390935,0.153846,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669


In [76]:
Final_df.to_csv('Final_features.csv',index=False)
Final_df

,ID,SEQUENCE,TARGETS,SEQ_LEN,MW,MIC,MIC_VALUE,UNIT,NEW-CONCENTRATION,A,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,DBAASP_6009,GLLNVIKDTAQNLFAAALEKLKCKVTKCN,Staphylococcus aureus ATCC 25923,29,3129.86,>160,160.000000,µM,2.204120,0.137931,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
1,dbAMP_14775,VPIIYCNRRTGKCQRM,Staphylococcus aureus,16,1920.40,(MIC=9µM),9.000000,µM,0.954243,0.000000,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
2,dbAMP_24894,TRSSRAGLQWPVGRVHRLLRK,Staphylococcus aureus,21,2455.93,(MIC=5µg/ml),2.035889,μg/ml,0.308754,0.047619,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
3,DBAASP_6975,KRRLILRILRLAIRALVKKR,Staphylococcus aureus 90857,20,2468.25,4,4.000000,µM,0.602060,0.100000,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
4,DBAASP_20056,IWWRRRWWI,Staphylococcus aureus ATCC 25923,9,1439.77,4,4.000000,µM,0.602060,0.000000,...,0.008224,0.015787,0.021713,0.013199,0.011265,0.019359,0.035320,0.020256,0.024270,0.044469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,DBAASP_5251,PRPLPFPRPGPRPI,Escherichia coli ATCC 25922,14,1578.96,>128,128.000000,µM,2.107210,0.000000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
8060,DBAASP_16141,KGGKFLNFLKKAAKVGAKVGMAALG,Escherichia coli NBRC 14237,25,2487.14,<3.125,3.125000,µM,0.494850,0.200000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
8061,dbAMP_25311,KARRAVRWI,Escherichia coli,9,1137.42,(MIC=1.2µg/ml),1.055019,μg/ml,0.023260,0.222222,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
8062,DBAASP_8600,GLLKRIKSLL,Escherichia coli ATCC 25922,10,1122.48,100,100.000000,µM,2.000000,0.000000,...,0.015460,0.011953,0.017996,0.020525,0.018351,0.012581,0.014832,0.018071,0.016588,0.023669
